<a href="https://colab.research.google.com/github/Sikandarh11/Computer-System-Architecture/blob/main/CNN_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!kaggle datasets download -d rizwan123456789/potato-disease-leaf-datasetpl


403 - Forbidden - Permission 'datasets.get' was denied


In [ ]:
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/potato-disease-leaf-datasetpld.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()


In [ ]:

import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

# Path to dataset directory
train_dir = '/content/PLD_3_Classes_256/Training'
validation_dir = '/content/PLD_3_Classes_256/Testing'

# Define data augmentation parameters
data_augmentation = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load training dataset with data augmentation and preprocessing
train_ds_augmented = data_augmentation.flow_from_directory(
    directory=train_dir,
    target_size=(256, 256),
    batch_size=8,
    class_mode='categorical'
)

# Load validation dataset
validation_ds_aug = data_augmentation.flow_from_directory(
    directory=validation_dir,
    target_size=(256, 256),
    batch_size=8,
    class_mode='categorical'
)

# Define a simpler model architecture
model = keras.Sequential([
    Conv2D(64, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(256, 256, 3)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2), strides=2, padding='same'),
    Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2), strides=2, padding='same'),
    Conv2D(16, kernel_size=(3, 3), padding='same', activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2), strides=2, padding='same'),
    Flatten(),
    Dense(32, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    Dense(3, activation='softmax')
])

# Compile the model with a smaller learning rate
model.compile(optimizer=Adam(learning_rate=0.0001), loss="categorical_crossentropy", metrics=['accuracy'])

# Train the model



Found 3251 images belonging to 3 classes.
Found 405 images belonging to 3 classes.


In [ ]:
history = model.fit(
    train_ds_augmented,
    steps_per_epoch=train_ds_augmented.samples // 8,
    validation_data=validation_ds_aug,
    validation_steps=validation_ds_aug.samples // 8,
    epochs=20
)


Epoch 1/20
406/406 [==============================] - 712s 2s/step - loss: 0.9610 - accuracy: 0.5809 - val_loss: 2.5989 - val_accuracy: 0.2525
Epoch 2/20
406/406 [==============================] - 712s 2s/step - loss: 0.7559 - accuracy: 0.6772 - val_loss: 1.1098 - val_accuracy: 0.5725
Epoch 3/20
406/406 [==============================] - 718s 2s/step - loss: 0.7119 - accuracy: 0.7068 - val_loss: 0.6369 - val_accuracy: 0.7575
Epoch 4/20
406/406 [==============================] - 691s 2s/step - loss: 0.6732 - accuracy: 0.7179 - val_loss: 0.5520 - val_accuracy: 0.7700
Epoch 5/20
406/406 [==============================] - 713s 2s/step - loss: 0.6227 - accuracy: 0.7404 - val_loss: 0.5078 - val_accuracy: 0.7675
Epoch 6/20
406/406 [==============================] - 688s 2s/step - loss: 0.5773 - accuracy: 0.7663 - val_loss: 0.4675 - val_accuracy: 0.8100
Epoch 7/20
406/406 [==============================] - 690s 2s/step - loss: 0.5831 - accuracy: 0.7669 - val_loss: 0.6729 - val_accuracy: 0.7675

In [ ]:
# Save the model
model.save("cnn_model_optimized.h5")

In [ ]:
import pickle
model.save("trained_model.h5")

# Save the training history
with open('training_history.pkl', 'wb') as f:
    pickle.dump(history.history, f)

In [4]:
from tensorflow.keras.models import load_model
# Load the model
model = load_model('/content/cnn_model_optimized.h5')

In [5]:
import zipfile
zip_ref = zipfile.ZipFile('/content/Testing.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create an ImageDataGenerator for preprocessing
test_datagen = ImageDataGenerator(rescale=1./255)

# Load and preprocess the testing images
test_generator = test_datagen.flow_from_directory(
    '/content/Testing',
    target_size=(256, 256),
    batch_size=8,
    class_mode='categorical'  # or 'binary' if it's a binary classification task
)

# Evaluate the model on the testing data
loss, accuracy = model.evaluate(test_generator)

print('Accuracy:', accuracy)


Found 405 images belonging to 3 classes.
51/51 [==============================] - 28s 521ms/step - loss: 0.4472 - accuracy: 0.8370
Accuracy: 0.8370370268821716
